In [23]:
import pandas as pd
import numpy as np
import re
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
df = pd.read_csv('AnalystJobs.csv')
df.head()
df.describe()

,Activelyrecruiting,Title,Company,Link,Experience Level,Employee Count/Domain,Description,Skills
count,650,975,975,975,975,961,833,424
unique,3,475,794,965,86,230,680,322
top,Actively recruiting,Data Analyst,Turing · India · Reposted 1 week ago,https://www.linkedin.com/jobs/view/3852553117/...,"On-site\nMatches your job preferences, workpla...","10,001+ employees · IT Services and IT Consulting",About the job\nExperience: 5.00+ years\n\nSala...,"['Data Analysis, Data Analytics, Documentation..."
freq,525,220,8,2,162,54,25,25


In [12]:
 df.duplicated().sum()

10

In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
# Seprate Column for Location & Company_Name from Company

df[['Company_Name', 'Location']] = df['Company'].str.split('·', expand=True)
df['Company_Name'] = split_data[0].str.strip()
df['Location'] = split_data[1].str.strip()

In [ ]:
# Seprate Column for Employee Size & Industry from Employee Count/Domain

split_data = df['Employee Count/Domain'].str.split('·', expand=True)
df['Employee Size'] = split_data[0].str.strip()
df['Industry'] = split_data[1].str.strip()

In [ ]:
df.loc[data['Employee Size'].notna() & data['Employee Size'].str.startswith('See'), 'Employee Size'] = None

In [ ]:
keywords = ['Internship', 'Entry level', 'Associate', 'Mid-Senior level', 'Director', 'Executive']

df['Experience'] = df1['Experience Level'].apply(lambda x: next((word for word in keywords if word in x), None))

In [ ]:
keywords = ['On-site', 'Remote', 'Hybrid']

df1['Work Mode'] = df1['Experience Level'].apply(lambda x: next((word for word in keywords if word in x), 'Other'))

In [ ]:
df.drop(columns=['Experience Level'], inplace=True)
df.drop(columns=['Employee Count/Domain'], inplace=True)
df.drop(columns=['Company_Name'], inplace=True)

In [ ]:
df['Activelyrecruiting'] = df['Activelyrecruiting'].replace('Actively recruiting', 'Yes')

In [ ]:
new_column_order = ['Link','Title', 'Skills', 'Description', 'Experience', 'Company_Name', 'Location',
                    'Employee Size', 'Industry', 'Work Mode', 'Activelyrecruiting']

df1 = df1.loc[:, new_column_order]

In [ ]:
df.convert_dtypes(infer_objects=True, convert_string=True, convert_integer=True, convert_boolean=True, convert_floating=True, dtype_backend='numpy_nullable')

In [ ]:
df1.rename(columns={
    'Title': 'Job Title',
    'Skills': 'Required Skills',
    'Description': 'Job Description',
    'Experience': 'Required Experience',
    'Company_Name': 'Company Name',
    'Location': 'Location',
    'Employee_Size': 'Employee Size',
    'Industry': 'Industry',
    'Work Mode': 'Work Mode'
    'Activelyrecruiting' : 'Actively recruiting'
})

In [ ]:
df['Required Skills'] = df['Required Skills'].str.replace(r"\['|'\]", "", regex=True)

In [ ]:
df['Location'] = df['Location'].str.replace('India', '').str.strip()

df[['City', 'State']] = df['Location'].str.split(',', expand=True)
df['City'] = df['City'].str.strip()
df['State'] = df['State'].str.strip()

In [ ]:
data.to_csv('modified_data', index=False)

In [36]:
df = pd.read_csv('modified_data.csv')

In [ ]:

df['Job Description'] = df['Job Description'].dropna(inplace=True)

def preprocess_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation and special characters
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenization
    tokens = word_tokenize(text)
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [word for word in tokens if word not in stop_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in filtered_tokens]
    return " ".join(lemmatized_tokens)

df['Cleaned_Job_Description'] = df['Job Description'].apply(preprocess_text)


# Assuming df['Cleaned_Job_Description'] contains preprocessed job descriptions
corpus = df['Cleaned_Job_Description']

In [ ]:
def extract_degrees(description):
    # Define regex patterns for degree keywords
    degree_patterns = {
        "bachelor": re.compile(r"(?i)\b(bachelor'?s?|bs|undergraduate)|b.e|bca|mca|btech\b"),
        "master": re.compile(r"(?i)\b(master'?s?|ms|graduate)\b"),
        "phd": re.compile(r"(?i)\b(phd|doctorate|doctoral|d\.? ?phil|ph\.? ?d\.?)\b")
    }

    # Initialize degree columns to 0
    degrees_present = {degree: 0 for degree in degree_patterns}

    # Search for degree patterns in the description
    for degree, pattern in degree_patterns.items():
        if re.search(pattern, description):
            degrees_present[degree] = 1

    return degrees_present

# Apply the function to each job description
degrees_df = corpus.apply(extract_degrees)

# Convert the result into a DataFrame
degrees_df = pd.DataFrame(degrees_df.tolist(), index=degrees_df.index)

# Concatenate the result with the original DataFrame
df = pd.concat([df, degrees_df], axis=1)

In [ ]:
df.to_csv('modified_data1.csv')

In [72]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

def tokenize(string):
    '''
    This function takes in a string and
    returns a tokenized string.
    '''
    tokenizer = nltk.tokenize.ToktokTokenizer()
    string = tokenizer.tokenize(string, return_str=True)
    return string

def remove_stopwords(string, extra_words=[], exclude_words=[]):
    '''
    This function takes in a string, optional extra_words and exclude_words parameters
    with default empty lists and returns a string.
    '''
    stopword_list = stopwords.words('english')
    stopword_list = set(stopword_list) - set(exclude_words)
    stopword_list = stopword_list.union(set(extra_words))

    words = string.split()
    filtered_words = [word for word in words if word not in stopword_list]
    string_without_stopwords = ' '.join(filtered_words)

    return string_without_stopwords

%%writefile process_skill.py

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
UsageError: Line magic function `%%writefile` not found.
